In [ ]:
! pip install promptsite langchain langchain_community openai

In [ ]:
import os
from pydantic import BaseModel, Field
from typing import List
from langchain.chat_models import ChatOpenAI
from promptsite.model.variable import ArrayVariable
from promptsite.decorator import tracker

In [ ]:
os.environ["OPENAI_API_KEY"]="your open ai key"

In [ ]:
# define input model
class SupportTicket(BaseModel):
    ticket_id: str = Field(..., description="Unique ticket identifier")
    customer_type: str = Field(..., enum=["free", "premium", "enterprise"])
    category: str = Field(..., enum=[
        "billing",
        "technical",
        "feature_request",
        "account_access",
        "bug_report"
    ])
    message: str = Field(..., description="Customer's message")
    language: str = Field(..., enum=["en", "es", "fr", "de"], description="Preferred language")
    previous_contacts: int = Field(..., description="Number of previous contacts", ge=0)

# define output model
class TicketResponse(BaseModel):
    ticket_id: str = Field(..., description="ID of the ticket being responded to")
    priority: str = Field(..., enum=["High", "Medium", "Low"])
    response: str = Field(..., description="Main response message in customer's language")
    next_steps: List[str] = Field(..., description="List of next steps or actions")
    documentation: str = Field(..., description="Relevant documentation links")


In [ ]:
template = """Generate appropriate customer support responses for the following tickets:

{{ tickets }}

For each ticket, provide:
1. A response in the customer's preferred language
2. Suggested next steps or resolution path
3. Any relevant documentation links
4. Priority level (High/Medium/Low)

Guidelines:
- Match tone to customer tier (formal for enterprise, friendly for others)
- Acknowledge repeat contacts with extra attention
- Include relevant policy information for billing issues
- Reference specific parts of their message


{{ output }}
"""

In [ ]:
# Usage with tracker decorator
@tracker(
    prompt_id="support_response_generator",
    description="Generates customer support responses based on ticket information",
    tags=["customer_support", "multilingual"],
    variables={
        "tickets": ArrayVariable(model=SupportTicket), 
        "output": ArrayVariable(model=TicketResponse, is_output=True)
    }
)
def generate_support_responses(content: str, llm_config: dict = {}, **kwargs):
    llm = ChatOpenAI(**llm_config)
    
    # Get response from LLM
    response = llm.invoke(content)
    
    # Return the response string
    return response.content



In [ ]:
tickets = [
    {
        "ticket_id": "T-1001",
        "customer_type": "enterprise",
        "category": "technical",
        "message": "API endpoints returning 503 errors for last 30 minutes",
        "language": "en",
        "previous_contacts": 0
    },
    {
        "ticket_id": "T-1003",
        "customer_type": "premium",
        "category": "feature_request",
        "message": "Can you add dark mode to the dashboard?",
        "language": "en",
        "previous_contacts": 1
    }
]


response = generate_support_responses(
    content=template,
    variables={"tickets": tickets},
    llm_config={
        "model_name": "gpt-4o-mini",
        "temperature": 0,
        "request_timeout": 100
    }
)

print(response)